<a href="https://colab.research.google.com/github/prat-amaze/Expiry_date_extractinator/blob/main/Subspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 7.0 MB/s 
     |████████████████████████████████| 619 kB 71.6 MB/s 
     |████████████████████████████████| 47.6 MB 114 kB/s 
     |████████████████████████████████| 145 kB 28.7 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [ ]:
import cv2
import easyocr
from pylab import rcParams
from PIL import Image
import json
import requests
rcParams['figure.figsize']= 8,16

In [ ]:
reader= easyocr.Reader(['en'])

In [ ]:
#importing date related libraries and modules
import re
from dateutil import parser
from dateutil.relativedelta import relativedelta
import datetime


In [ ]:
#function for converting text
def image_to_text(url):
  try:
    img= Image.open(requests.get(url, stream=True).raw)
    text=reader.readtext(img, detail=0)
    return text
  except:
    return "f"


In [ ]:
#loading dataset
pk = requests.get("https://cdn.subspace.money/internship/data_load.json")
dataset = pk.json()

In [ ]:
def transform_data(dataset):
  date= [] #list of all expiry dates
  for i, o in enumerate(dataset):
    created= parser.parse(o["created_at"]).date()
    #getting values of duration and created at
    if(o["whatsub_plan"]["duration_type"]=="months"):
      dur= (o["whatsub_plan"]["duration"])
    else:
      dur=12*(o["whatsub_plan"]["duration"])
    #reading image and converting to text
    image_url = o["expiry_image"]
    image_text = image_to_text(image_url)
    z= " ".join(image_text)

    image_date=re.findall("[ADFJMNOS]\w*[\W]+\d{2,4}[\W]+\d{2,4}|\d{2,4}[\W]+[ADFJMNOS]\w*[\W]+\d{2,4}|\d{2,4}[\W]+\d{2,4}[\W]+\d{2,4}|\d{2,4}[\W]+\d{2,4}[\W]+[ADFJMNOS]\w*", z)
    #converting all dates in text to datetime.date type
    a=[]
    for j in image_date:
      try:
        d= parser.parse(j, fuzzy= True).date()
        print(type(d))
        a.append(d)
      except:
        pass
    #checking correct date before adding in dates
    if(len(a)==1):
      x = a[0].strftime('%Y-%m-%d')
      date.append(x)
    elif(len(a)>1):
      l= created + relativedelta(dur)
      if((a[0]- l) < (a[1]-l)):
        x = a[0].strftime('%Y-%m-%d')
        date.append(x)
      else:
        x = a[1].strftime('%Y-%m-%d')
        date.append(x)
    else:
      pass

  return date

In [ ]:
expiry_date= transform_data(dataset[0:5])

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>


In [ ]:
expiry_date

['2020-01-21', '2022-03-29', '2022-12-27']